In [80]:
import pandas as pd

In [81]:
path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/summary.xlsx'

In [90]:
raw = pd.read_excel("/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/ATB Raw Data.xlsx").drop(axis='columns', columns=['core_metric_case', 'crpyears', 'technology', 'techdetail', 'default']).drop_duplicates(keep='first')
raw

,core_metric_key,core_metric_parameter,technology_alias,display_name,scenario,core_metric_variable,units,value
0,MA*CoFECCS90F,CAPEX,Coal,Coal-90%-CCS,Advanced,2020,$/kW,5541.600067
1,MA*CoFECCS90F,CAPEX,Coal,Coal-90%-CCS,Advanced,2021,$/kW,5541.600067
2,MA*CoFECCS90F,CAPEX,Coal,Coal-90%-CCS,Advanced,2022,$/kW,5170.403865
3,MA*CoFECCS90F,CAPEX,Coal,Coal-90%-CCS,Advanced,2023,$/kW,5130.889431
4,MA*CoFECCS90F,CAPEX,Coal,Coal-90%-CCS,Advanced,2024,$/kW,5091.374996
...,...,...,...,...,...,...,...,...
97552,MM6NuarNucler,Variable O&M,Nuclear,Nuclear - AP1000,Moderate,2046,$/MWh,2.840000
97553,MM6NuarNucler,Variable O&M,Nuclear,Nuclear - AP1000,Moderate,2047,$/MWh,2.840000
97554,MM6NuarNucler,Variable O&M,Nuclear,Nuclear - AP1000,Moderate,2048,$/MWh,2.840000
97555,MM6NuarNucler,Variable O&M,Nuclear,Nuclear - AP1000,Moderate,2049,$/MWh,2.840000


## Join display name groups under same alias having same values
1. iterate over alias names
2. iterate over years
3.

In [99]:
# get all unique alias names
alias = list(raw['technology_alias'].unique())

# iterate over different alias names
for name in alias:
    flag = True

    # select the subset of data with same alias names
    alias_df = raw[raw['technology_alias']==name]

    # get all unique year
    years = list(alias_df['core_metric_variable'].unique())

    # iterate over each year
    for year in years:
        if not flag:
            break
        # get subset of data
        years_df = alias_df[alias_df['core_metric_variable']==year]

        # get all core metric parameter names
        metric_paras = list(years_df['core_metric_parameter'].unique())

        # iterate over parameters
        for p in metric_paras:
            if not flag:
                break
            # get subset of data
            paras_df = years_df[years_df['core_metric_parameter']==p]

            # if all values are the same
            if not (paras_df['value']==paras_df['value'].iloc[0]).all():
                flag=False
                break

    if flag:
        print(name)
        raw.loc[raw['technology_alias']==name, 'display_name'] = name

raw.drop_duplicates(keep='first', inplace=True, subset=['core_metric_parameter', 'technology_alias', 'core_metric_variable', 'value'])

# raw

KeyboardInterrupt: 

## Extract baseline data (2020) and save to sheet

1. query the row of different 'display name' and of year 2020
2. try to join the ones under same alias with same value; that is, when all rows of different display names under the same alias are having same value in year 2020, we join these rows;

In [83]:
# select the subset of data
baseline = raw[(raw['core_metric_variable']==2020)&(raw['units']=='$/kW')][['technology_alias', 'display_name', 'value']].drop_duplicates(keep='first').reset_index(drop=True)

baseline

,technology_alias,display_name,value
0,Coal,Coal-90%-CCS,5541.600067
1,Coal,Coal-IGCC,5327.264195
2,Coal,Coal,3074.941437
3,Commercial Battery Storage,Commercial Battery Storage 1Hr,1251.613753
4,Commercial Battery Storage,Commercial Battery Storage 2Hr,1575.158689
...,...,...,...
192,Offshore Wind,Offshore Wind - Class 5,3953.353364
193,Offshore Wind,Offshore Wind - Class 6,3925.353262
194,Offshore Wind,Offshore Wind - Class 7,3960.704072
195,Offshore Wind,Offshore Wind - Class 8,4896.170393


In [85]:
# get all unique alias names
alias = list(baseline['technology_alias'].unique())

# iterate over all alias
for name in alias:
    df = baseline[baseline['technology_alias']==name]
    # if all values under the same alias group are the same
    if (df['value']==df['value'].iloc[0]).all():
        baseline.loc[baseline['technology_alias']==name, 'display_name'] = name

baseline = baseline.drop_duplicates(keep='first')

# write the baseline data to the baseline sheet of the file
baseline[['display_name', 'value']].to_excel(path, sheet_name='baseline', index=False)

# give you a look
baseline[['display_name', 'value']]

## Generate summary sheet for each `core_metric_parameter`

1. iterate over each `core_metric_parameter`
2. convert data from longform to wideform

In [87]:
# get all unique parameters
metric_paras = list(raw['core_metric_parameter'].unique())

# iterate over metric parameters
for p in metric_paras:
    df = raw[raw['core_metric_parameter'] == p].drop(axis='columns', columns=['core_metric_parameter'])
    df = df.pivot(index='core_metric_variable', columns=)

In [105]:
a = raw['display_name'].unique()